# Selenium 동적 페이지 다루기
- 동적 페이지?:
접속될 때 동작을 하는 것이 아니라 사용자가 동작을 했을 때 불려와지는 것.<br>
ex) 스크롤을 내렸을 때 새로운 목록 갱신.

## Google Moive
인기차트 목록 중 할인하고 있는 작품의 정보 가져오기.
#### 먼저, bs4로 가져와보기

In [8]:
import requests
from bs4 import BeautifulSoup

url = "https://play.google.com/store/movies/top"
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language":"ko-KR,ko"      } #한국어 페이지로 가져오기 위해

res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text,"lxml")

movies = soup.find_all("div", attrs = {"class": "ImZGtf mpg5gc"}) 
print(len(movies))

for movie in movies:
    title = movie.find("div", attrs={"class": "WsMG1c nnK0zc"}).get_text()
    print(title)
    


10
노바디
어벤져스 : 엔드게임 (자막판)
콰이어트 플레이스 (자막판)
캐시트럭
Cruella
고질라 VS. 콩
컨저링 3: 악마가 시켰다
어벤져스 : 인피니티 워 (자막판)
크루즈 패밀리: 뉴 에이지
너의 이름은. (자막판)


## Selenium을 이용해서 동적 페이지 다루기
#### 스크롤 

In [9]:
from selenium import webdriver
browser = webdriver.Chrome()
browser.maximize_window()

#페이지 이동
url = "https://play.google.com/store/movies/top"
browser.get(url)

#지정한 위치로 스크롤 내리기
#모니터(해상도) 높이의 1080으로 스크롤 내리기
#browser.execute_script("window.scrollTo(0,1080)") #스크롤을 세로방향으로 1080 내려라
#browser.execute_script("window.scrollTo(0,2080)") # 더 스크롤하기

# 화면 가장 아래로 스크롤 내리기
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)") #현 문서의 가장 아래쪽으로

import time
interval = 5 #2초에 한번씩 스크롤 내림.

#현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script("return document.body.scrollHeight")

#반복 수행
while True:
    #스크롤 가장 아래로 내림
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    #페이지 로딩 대기
    time.sleep(interval)
    
    #현재 문서 높이를 가져와서 저장
    curr_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    if curr_height == prev_height:
        break
    
    prev_height = curr_height
    
print("스크롤 완료")
    

스크롤 완료


### 스크래핑 시작

In [10]:
import requests
from bs4 import BeautifulSoup


soup = BeautifulSoup(browser.page_source,"lxml")

#movies = soup.find_all("div", attrs = {"class":["ImZGtf mpg5gc", "Vpfmgd"]}) 
movies = soup.find_all("div", attrs = {"class": "Vpfmgd"})
print(len(movies))

for movie in movies:
    title = movie.find("div", attrs={"class": "WsMG1c nnK0zc"}).get_text()
    # 할인 전 가격
    original_price = movie.find("span", attrs={"class": "SUZt4c djCuy"})
    if original_price:
        original_price = original_price.get_text()
        
    else:
        #print(title, "<할인되지 않은 영화 제외>")
        continue
    
    #할인된 가격
    price = movie.find("span", attrs={"class":"VfPpfd ZdBevf i5DZme"})
    
    #링크 
    link = movie.find("a", attrs={"class":"JC71ub"})["href"]
    #올바른 링크: https://play.google.com +link
    
    print(f"제목: {title}")
    print(f"할인 전 금액: {original_price}")
    print(f"할인 후 금액: {price}")
    print("링크:", "https://play.google.com" + link)
    print("-"*120)
    
browser.quit()

160
제목: 컨저링 3: 악마가 시켰다
할인 전 금액: ₩14,900
할인 후 금액: <span class="VfPpfd ZdBevf i5DZme"><span>₩9,900</span></span>
링크: https://play.google.com/store/movies/details/%EC%BB%A8%EC%A0%80%EB%A7%81_3_%EC%95%85%EB%A7%88%EA%B0%80_%EC%8B%9C%EC%BC%B0%EB%8B%A4?id=jhaUnwBzVBM.P
------------------------------------------------------------------------------------------------------------------------
제목: 소울
할인 전 금액: ₩4,400
할인 후 금액: <span class="VfPpfd ZdBevf i5DZme"><span>₩2,750</span></span>
링크: https://play.google.com/store/movies/details/%EC%86%8C%EC%9A%B8?id=fRcet-N-fIs.P
------------------------------------------------------------------------------------------------------------------------
제목: 스파이더맨: 파 프롬 홈 (자막판)
할인 전 금액: ₩1,800
할인 후 금액: <span class="VfPpfd ZdBevf i5DZme"><span>₩900</span></span>
링크: https://play.google.com/store/movies/details/%EC%8A%A4%ED%8C%8C%EC%9D%B4%EB%8D%94%EB%A7%A8_%ED%8C%8C_%ED%94%84%EB%A1%AC_%ED%99%88_%EC%9E%90%EB%A7%89%ED%8C%90?id=zsehQ83TtjI
------------------------------